# Double-Cross Strategy

## Background

### Pairing the Stochastic and MACD
Looking for two popular indicators that work well together resulted in this pairing of the stochastic oscillator and the moving average convergence divergence (MACD). This team works because the stochastic is comparing a stock's closing price to its price range over a certain period of time, while the MACD is the formation of two moving averages diverging from and converging with each other. This dynamic combination is highly effective if used to its fullest potential.

https://www.investopedia.com/articles/trading/08/macd-stochastic-double-cross.asp

In [1]:
import time as time
import datetime as datetime

import backtesting
import pandas as pd
import yfinance as yf
import matplotlib as plt
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

/usr/local/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#Ticker in url
ticker = 'AAPL'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2018, 12, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2022, 12, 31, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history'

In [3]:
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

In [4]:
#index data frame to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [5]:
#dropping columns that are not necessary
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2018-12-03,46.115002,46.235001,45.302502,46.205002,163210000
2018-12-04,45.237499,45.597500,44.067501,44.172501,165377200
2018-12-06,42.939999,43.695000,42.605000,43.680000,172393600
2018-12-07,43.372501,43.622501,42.075001,42.122501,169126400
2018-12-10,41.250000,42.522499,40.832500,42.400002,248104000
...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,85438400
2022-12-29,127.989998,130.479996,127.730003,129.610001,75703700


In [ ]:
#the overall trading strategy function
class RSI(Strategy):
    #define upper and lower bands of indicator. If the stock price
    #dips below 30 it buys and above 70 it sells
    upper_bound = 70
    lower_bound = 30

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, 14)

    def next(self):

        if crossover(self.rsi, self.upper_bound):
            #if this statement is true the below command signals a sell.
            self.sell()

        elif crossover(self.lower_bound, self.rsi):
            #buy command
            self.buy()


#bt variable runs the backtest dependent on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to
#backtesting.py on github
bt = Backtest(df, RSI, cash = 1_000)

stats = bt.run()
stats

In [ ]:
#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
bt.plot()